<a href="https://colab.research.google.com/github/PhazeRoOman/DeepDTA/blob/master/Implementing_DeepDTA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/PhazeRo/DeepDTA_Implementation/source
%ls

/content/drive/MyDrive/PhazeRo/DeepDTA_Implementation/source
arguments.py   go.sh                       new_rep_davis_backup.mat
auc.jar        Implementing_DeepDTA.ipynb  new_rep_kiba_backup.mat
data/          indices_unique_davis.mat    __pycache__/
datahelper.py  indices_unique_kiba.mat     run_experiments_original.py
emetrics.py    logs/                       run_experiments.py
figures/       model_deepDTA.keras


/content/drive/MyDrive/PhazeRo/DeepDTA_Implementation/source

In [3]:
!pip install matplotlib==3.2.2
# !pip install transformers

In [4]:
%pip show matplotlib
# %pip show transformers

Name: matplotlib
Version: 3.2.2
Summary: Python plotting package
Home-page: https://matplotlib.org
Author: John D. Hunter, Michael Droettboom
Author-email: matplotlib-users@python.org
License: PSF
Location: /usr/local/lib/python3.10/dist-packages
Requires: cycler, kiwisolver, numpy, pyparsing, python-dateutil
Required-by: arviz, datascience, fastai, flax, geemap, imgaug, matplotlib-venn, missingno, mizani, mlxtend, music21, plotnine, prophet, pycocotools, seaborn, wordcloud, yellowbrick


In [5]:
from __future__ import print_function

from collections import namedtuple

import numpy as np
import tensorflow as tf
import random as rn

import os
os.environ['PYTHONHASHSEED'] = '0'

np.random.seed(1)
rn.seed(1)

session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)

import keras
from keras import backend as K
tf.compat.v1.keras.backend.set_session
tf.random.set_seed(0)

from itertools import product

In [6]:
from datahelper import *
from arguments import argparser, logging

In [7]:
import keras
from keras.models import Model
from keras.preprocessing import sequence
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D
from keras.layers import BatchNormalization
from keras.layers import Conv2D, GRU
from keras.layers import Input, Embedding, LSTM, Dense, TimeDistributed, Masking, RepeatVector, Flatten
from keras.models import Model
from keras.utils import plot_model
from keras.layers import Bidirectional
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import optimizers, layers
from keras.optimizers import Adam


In [8]:
import sys, pickle, os
import math, json, time
import decimal
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from random import shuffle
from copy import deepcopy
from sklearn import preprocessing
from emetrics import get_aupr, get_cindex, get_rm2

In [9]:
TABSY = "\t"
figdir = "figures/"

In [10]:
def build_combined_categorical(FLAGS, NUM_FILTERS, FILTER_LENGTH, learning_rate):

    # FLAGS.num_windows = [32]  # num_filters
    # FLAGS.seq_window_lengths = [4, 8, 12]#[8,12]  # kernel_size
    # FLAGS.smi_window_lengths = [4]#[4,8]

    XDinput = Input(shape=(FLAGS.max_smi_len,), dtype='int32') ### Buralar flagdan gelmeliii
    XTinput = Input(shape=(FLAGS.max_seq_len,), dtype='int32')

    print('NUM_FILTERS:', NUM_FILTERS)
    print('FILTER_LENGTH:', FILTER_LENGTH)
    print('learning_rate:', learning_rate)
    ### SMI_EMB_DINMS  FLAGS GELMELII
    encode_smiles = Embedding(input_dim=FLAGS.charsmiset_size+1, output_dim=128, input_length=FLAGS.max_smi_len)(XDinput)
    encode_smiles = Conv1D(filters=NUM_FILTERS, kernel_size=FILTER_LENGTH,  activation='relu', padding='valid',  strides=1)(encode_smiles)
    encode_smiles = Conv1D(filters=NUM_FILTERS*2, kernel_size=FILTER_LENGTH,  activation='relu', padding='valid',  strides=1)(encode_smiles)
    encode_smiles = Conv1D(filters=NUM_FILTERS*3, kernel_size=FILTER_LENGTH,  activation='relu', padding='valid',  strides=1)(encode_smiles)
    encode_smiles = GlobalMaxPooling1D()(encode_smiles)

    encode_protein = Embedding(input_dim=FLAGS.charseqset_size+1, output_dim=128, input_length=FLAGS.max_seq_len)(XTinput)
    encode_protein = Conv1D(filters=NUM_FILTERS, kernel_size=FILTER_LENGTH,  activation='relu', padding='valid',  strides=1)(encode_protein)
    encode_protein = Conv1D(filters=NUM_FILTERS*2, kernel_size=FILTER_LENGTH,  activation='relu', padding='valid',  strides=1)(encode_protein)
    encode_protein = Conv1D(filters=NUM_FILTERS*3, kernel_size=FILTER_LENGTH,  activation='relu', padding='valid',  strides=1)(encode_protein)
    encode_protein = GlobalMaxPooling1D()(encode_protein)

    encode_interaction = keras.layers.concatenate([encode_smiles, encode_protein], axis=-1) #merge.Add()([encode_smiles, encode_protein])

    # Fully connected
    FC1 = Dense(1024, activation='relu')(encode_interaction)
    FC2 = Dropout(0.1)(FC1)
    FC2 = Dense(1024, activation='relu')(FC2)
    FC2 = Dropout(0.1)(FC2)
    FC2 = Dense(512, activation='relu')(FC2)

    # And add a logistic regression on top
    predictions = Dense(1, kernel_initializer='normal')(FC2) #OR no activation, rght now it's between 0-1, do I want this??? activation='sigmoid'

    interactionModel = Model(inputs=[XDinput, XTinput], outputs=[predictions])

    adam=Adam(learning_rate=learning_rate)
    interactionModel.compile(optimizer=adam, loss='mean_squared_error', metrics=[cindex_score]) #, metrics=['cindex_score']
    # print(interactionModel.summary())
    plot_model(interactionModel, to_file='figures/build_combined_categorical.png')

    return interactionModel

In [11]:
def nfold_1_2_3_setting_sample(XD, XT,  Y, label_row_inds, label_col_inds, measure, runmethod,  FLAGS, dataset):

    # bestparamlist = []
    test_set, outer_train_sets = dataset.read_sets(FLAGS)

    # print('\n')
    # print('*Printing in nfold_1_2_3_setting_sample*\n')
    # print('test_set:', test_set)
    # print('max(test_set):', max(test_set))
    # print('len(outer_train_sets):', len(outer_train_sets))
    # print('max(max(outer_train_sets)):', max(max(outer_train_sets)))
    # print('outer_train_sets:', outer_train_sets)

    foldinds = len(outer_train_sets)

    test_sets = []
    ## TRAIN AND VAL
    val_sets = []
    train_sets = []

    #logger.info('Start training')
    for val_foldind in range(foldinds):
        val_fold = outer_train_sets[val_foldind]
        # print('val_foldind:', val_foldind)
        # print('val_fold:', val_fold)
        val_sets.append(val_fold)
        otherfolds = deepcopy(outer_train_sets)
        otherfolds.pop(val_foldind)
        otherfoldsinds = [item for sublist in otherfolds for item in sublist]
        train_sets.append(otherfoldsinds)
        # test_sets.append(test_set)
        print("val set", str(len(val_fold)))
        print("train set", str(len(otherfoldsinds)))
    test_sets.append(test_set)



    # bestparamind, best_param_list, bestperf, all_predictions_not_need, \
    # losses_not_need, trained_model = general_nfold_cv(XD, XT,  Y, label_row_inds, \
    #                                                   label_col_inds, measure, \
    #                                                   runmethod, FLAGS, train_sets, \
    #                                                   val_sets)

    best_parameters = []
    bestparamind, best_param_list, bestperf, all_predictions_not_need, \
    losses_not_need, trained_model = general_nfold_cv(XD, XT,  Y,
                                                      label_row_inds, label_col_inds,
                                                      measure,
                                                      runmethod,
                                                      FLAGS,
                                                      train_sets, val_sets,
                                                      best_parameters,
                                                      TUNING_FLAG=1,
                                                      FITTING_FLAG=1)

    bestparamind = 0
    best_parameters = [FLAGS.num_windows[best_param_list[0]],
                       FLAGS.smi_window_lengths[best_param_list[1]],
                       FLAGS.seq_window_lengths[best_param_list[2]]]


    #print("Test Set len", str(len(test_set)))
    #print("Outer Train Set len", str(len(outer_train_sets)))
    bestparam, best_param_list, bestperf, all_predictions, \
    all_losses, trained_model = general_nfold_cv(XD, XT, Y,
                                                 label_row_inds, label_col_inds,
                                                 measure,
                                                 runmethod,
                                                 FLAGS,
                                                 train_sets, val_sets,
                                                 best_parameters,
                                                 TUNING_FLAG=0,
                                                 FITTING_FLAG=1)

    valperfs = []
    valloss= []
    avgperf = 0.

    for val_foldind in range(len(val_sets)):
        foldperf = all_predictions[bestparamind][val_foldind]
        foldloss = all_losses[bestparamind][val_foldind]
        valperfs.append(foldperf)
        valloss.append(foldloss)
        avgperf += foldperf

    avgperf = avgperf / len(val_sets)
    avgloss = np.mean(valloss)
    valstd = np.std(valperfs)


    bestparam, best_param_list, bestperf, all_predictions, \
    all_losses = general_nfold_cv(XD, XT, Y,
                                  label_row_inds, label_col_inds,
                                  measure,
                                  trained_model,
                                  FLAGS,
                                  train_sets, test_sets,
                                  best_parameters,
                                  TUNING_FLAG=0,
                                  FITTING_FLAG=0)


    # print('all_predictions:', all_predictions)

    logging("---FINAL RESULTS-----", FLAGS)
    logging("best param index = %s,  best param = %.5f" %
            (bestparamind, bestparam), FLAGS)

    testperfs = bestperf
    testloss = all_losses[0]

    # testperfs = []
    # testloss= []

    # avgperf = 0.

    # for test_foldind in range(len(test_sets)):
    #   for test_foldind in range(len(test_set)):
    #     foldperf = all_predictions[bestparamind][test_foldind]
    #     foldloss = all_losses[bestparamind][test_foldind]
    #     testperfs.append(foldperf)
    #     testloss.append(foldloss)
    #     avgperf += foldperf

    # avgperf = avgperf / len(test_sets)
    # avgloss = np.mean(testloss)
    # teststd = np.std(testperfs)

    logging("Test Performance CI", FLAGS)
    logging(testperfs, FLAGS)
    logging("Test Performance MSE", FLAGS)
    logging(testloss, FLAGS)

    # return avgperf, avgloss, teststd, trained_model
    return avgperf, avgloss, valstd, trained_model

In [12]:
def general_nfold_cv(XD, XT,  Y, label_row_inds, label_col_inds, prfmeasure, runmethod, FLAGS, labeled_sets, val_sets, best_parameters, TUNING_FLAG, FITTING_FLAG): ## BURAYA DA FLAGS LAZIM????

    if FITTING_FLAG != 1 and FITTING_FLAG != 0:
      raise Exception('Error! FITTING_FLAG should be 1 or 0.')

    if TUNING_FLAG == 1 and FITTING_FLAG == 0:
      raise Exception('Error! TUNING_FLAG = 1 and FITTING_FLAG = 0. Are you sure?')

    if TUNING_FLAG == 1:
      paramset1 = FLAGS.num_windows                              #[32]#[32,  512] #[32, 128]  # filter numbers
      paramset2 = FLAGS.smi_window_lengths                               #[4, 8]#[4,  32] #[4,  8] #filter length smi
      paramset3 = FLAGS.seq_window_lengths                               #[8, 12]#[64,  256] #[64, 192]#[8, 192, 384]
      PRINTING_FLAG = 'Tuning'
    elif TUNING_FLAG == 0:
      if FITTING_FLAG == 1:
        PRINTING_FLAG = 'Final training'
      elif FITTING_FLAG == 0:
        PRINTING_FLAG = 'Evaluation only'
      paramset1 = [best_parameters[0]]
      paramset2 = [best_parameters[1]]
      paramset3 = [best_parameters[2]]
    else:
      raise Exception('Error! Tuning flag should be 1 or 0.')

    epoch = FLAGS.num_epoch                                 #100
    batchsz = FLAGS.batch_size                             #256

    logging("---Parameter Search-----", FLAGS)

    w = len(val_sets)
    h = len(paramset1) * len(paramset2) * len(paramset3)

    all_predictions = [[0 for x in range(w)] for y in range(h)]
    all_losses = [[0 for x in range(w)] for y in range(h)]

    # print('all_predictions:',all_predictions)

    # print('len(val_sets):', len(val_sets))

    print('\n')
    print('===========================================================')
    print(PRINTING_FLAG)
    print('Number of runs:', w*h)
    print('Number of folds:', w)
    print('Number of hyperparameters combinations for each fold:', h)
    print('===========================================================')

    runs_counter = 1

    # if FITTING_FLAG == 0:
    #   val_sets_length = 1
    # else:
    #   val_sets_length = len(val_sets)
    for foldind in range(len(val_sets)):
        # print()
        # print('*Printing in general_nfold_cv*\n')
        # print('foldind:', foldind)
        # print('val_sets:', val_sets)
        valinds = val_sets[foldind]
        # print('valinds = val_sets[foldind]:', valinds)
        # print('labeled_sets:', labeled_sets)
        labeledinds = labeled_sets[foldind]
        # print('labeledinds = labeled_sets[foldind]:', labeledinds)

        Y_train = np.mat(np.copy(Y))

        params = {}
        XD_train = XD
        XT_train = XT
        # print()
        # print('labeledinds:', labeledinds)
        # print('label_row_inds:', label_row_inds)
        # print('label_col_inds:', label_col_inds)
        # print()
        trrows = label_row_inds[labeledinds]
        trcols = label_col_inds[labeledinds]
        # print()
        # print('******* foldind:', foldind)
        # print('len(trrows):', len(trrows))
        # print('len(trcols):', len(trcols))
        # print('trrows', trrows)
        # print('trcols:', trcols)
        # print('len(label_row_inds):', len(label_row_inds))
        # print('len(label_col_inds):', len(label_col_inds))

        XD_train = XD[trrows]
        XT_train = XT[trcols]
        # print()
        # print('XD_train.shape:',XD_train.shape)
        # print('XT_train.shape:',XD_train.shape)
        # print()
        # print('right before prepare_interaction_pairs for train:')
        # print('XD.shape:', XD.shape)
        # print('XT.shape:', XT.shape)
        # print('Y.shape:', Y.shape)

        train_drugs, train_prots,  train_Y = prepare_interaction_pairs(XD, XT, Y, trrows, trcols)

        # print()
        # print('train_drugs.shape:',train_drugs.shape, type(train_drugs))
        # print('train_prots.shape:',train_prots.shape, type(train_prots))
        # print('len(train_Y):', len(train_Y), type(train_Y))

        terows = label_row_inds[valinds]
        tecols = label_col_inds[valinds]
        # print()
        # print('len(terows):', len(terows))
        # print('len(tecols):', len(tecols))
        # print('terows', terows)
        # print('tecols:', tecols)
        # print('len(label_row_inds):', len(label_row_inds))
        # print('len(label_col_inds):', len(label_col_inds))
        #print("terows", str(terows), str(len(terows)))
        #print("tecols", str(tecols), str(len(tecols)))

        val_drugs, val_prots,  val_Y = prepare_interaction_pairs(XD, XT,  Y, terows, tecols)

        # print()
        # print('val_drugs.shape:', val_drugs.shape, type(val_drugs))
        # print('val_prots.shape:', val_prots.shape, type(val_prots))
        # print('len(val_Y):', len(val_Y), type(val_Y))
        # print()

        pointer = 0
        # print('len(paramset1):',len(paramset1))
        # print('len(paramset2):',len(paramset2))
        # print('len(paramset3):',len(paramset3))
        for param1ind in range(len(paramset1)): #hidden neurons
            param1value = paramset1[param1ind]
            for param2ind in range(len(paramset2)): #learning rate
                param2value = paramset2[param2ind]
                for param3ind in range(len(paramset3)):
                    param3value = paramset3[param3ind]

                    # deepmethod = build_combined_categorical
                    # build_combined_categorical(FLAGS, NUM_FILTERS, FILTER_LENGTH1, FILTER_LENGTH2)
                    # paramset1 = FLAGS.num_windows                              #[32]#[32,  512] #[32, 128]  # filter numbers
                    # paramset2 = FLAGS.smi_window_lengths                               #[4, 8]#[4,  32] #[4,  8] #filter length smi
                    # paramset3 = FLAGS.seq_window_lengths                               #[8, 12]#[64,  256] #[64, 192]#[8, 192, 384]
                    print()
                    print(f'runs_counter: {runs_counter}/{w*h} --> {PRINTING_FLAG}.')
                    runs_counter += 1
                    print('fold #:', foldind+1)
                    if FITTING_FLAG == 1:
                      gridmodel = runmethod(FLAGS, param1value, param2value, param3value)
                      es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=15)
                      gridres = gridmodel.fit(([np.array(train_drugs),np.array(train_prots) ]), np.array(train_Y), batch_size=batchsz, epochs=epoch,
                                              validation_data=( ([np.array(val_drugs), np.array(val_prots) ]), np.array(val_Y)),  shuffle=False, callbacks=[es] )

                    elif FITTING_FLAG == 0:
                      gridmodel = runmethod

                    predicted_labels = gridmodel.predict([np.array(val_drugs), np.array(val_prots) ])
                    loss, rperf2 = gridmodel.evaluate(([np.array(val_drugs),np.array(val_prots) ]), np.array(val_Y), verbose=0)
                    rperf = prfmeasure(val_Y, predicted_labels)
                    rperf = rperf[0]

                    logging("P1 = %d,  P2 = %d, P3 = %d, Fold = %d, CI-i = %f, CI-ii = %f, MSE = %f" %
                    (param1ind, param2ind, param3ind, foldind, rperf, rperf2, loss), FLAGS)

                    if FITTING_FLAG == 1:
                      plotLoss(gridres, param1ind, param2ind, param3ind, foldind, TUNING_FLAG)

                    all_predictions[pointer][foldind] = rperf #TODO FOR EACH VAL SET allpredictions[pointer][foldind]
                    all_losses[pointer][foldind]= loss

                    pointer +=1

    bestperf = -float('Inf')
    bestpointer = None


    best_param_list = []
    ##Take average according to folds, then chooose best params
    pointer = 0
    for param1ind in range(len(paramset1)):
            for param2ind in range(len(paramset2)):
                for param3ind in range(len(paramset3)):
                    avgperf = 0.
                    for foldind in range(len(val_sets)):
                        foldperf = all_predictions[pointer][foldind]
                        avgperf += foldperf
                    avgperf /= len(val_sets)
                    #print(epoch, batchsz, avgperf)
                    if avgperf > bestperf:
                        bestperf = avgperf
                        bestpointer = pointer
                        best_param_list = [param1ind, param2ind, param3ind]

                    pointer +=1

    if FITTING_FLAG == 1:
      return  bestpointer, best_param_list, bestperf, all_predictions, all_losses, gridmodel
    elif FITTING_FLAG == 0:
      return  bestpointer, best_param_list, bestperf, all_predictions, all_losses

In [13]:
def cindex_score(y_true, y_pred):

    g = tf.subtract(tf.expand_dims(y_pred, -1), y_pred)
    g = tf.cast(g == 0.0, tf.float32) * 0.5 + tf.cast(g > 0.0, tf.float32)

    f = tf.subtract(tf.expand_dims(y_true, -1), y_true) > 0.0
    f = tf.compat.v1.matrix_band_part(tf.cast(f, tf.float32), -1, 0)

    g = tf.reduce_sum(tf.multiply(g, f))
    f = tf.reduce_sum(f)

    return tf.where(tf.equal(g, 0), 0.0, g/f) #select

In [14]:
def plotLoss(history, batchind, epochind, param3ind, foldind, TUNING_FLAG):

    if TUNING_FLAG:
      figname = "b"+str(batchind) + "_e" + str(epochind) + "_" + str(param3ind) + "_"  + str( foldind) + "_" + str(time.time())
    else:
      figname = "FINAL_TUNED_b"+str(batchind) + "_e" + str(epochind) + "_" + str(param3ind) + "_"  + str(foldind) + "_" + str(time.time())

    plt.figure()
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
	#plt.legend(['trainloss', 'valloss', 'cindex', 'valcindex'], loc='upper left')
    plt.legend(['trainloss', 'valloss'], loc='upper left')
    plt.savefig("figures/"+figname +".png" , dpi=None, facecolor='w', edgecolor='w', orientation='portrait',
                    papertype=None, format=None,transparent=False, bbox_inches=None, pad_inches=0.1)#,frameon=None)
    plt.close()


    ## PLOT CINDEX
    plt.figure()
    plt.title('model concordance index')
    plt.ylabel('cindex')
    plt.xlabel('epoch')
    plt.plot(history.history['cindex_score'])
    plt.plot(history.history['val_cindex_score'])
    plt.legend(['traincindex', 'valcindex'], loc='upper left')
    plt.savefig("figures/"+figname + "_acc.png" , dpi=None, facecolor='w', edgecolor='w', orientation='portrait',
                            papertype=None, format=None,transparent=False, bbox_inches=None, pad_inches=0.1)#,frameon=None)
    plt.close()

In [15]:
def prepare_interaction_pairs(XD, XT,  Y, rows, cols):
    drugs = []
    targets = []
    targetscls = []
    affinity=[]

    for pair_ind in range(len(rows)):
        drug = XD[rows[pair_ind]]
        drugs.append(drug)

        target=XT[cols[pair_ind]]
        targets.append(target)

        affinity.append(Y[rows[pair_ind],cols[pair_ind]])

    drug_data = np.stack(drugs)
    target_data = np.stack(targets)

    return drug_data, target_data, affinity

In [16]:
def experiment(FLAGS, perfmeasure, deepmethod, foldcount=6): #5-fold cross validation + test

    #Input
    #XD: [drugs, features] sized array (features may also be similarities with other drugs
    #XT: [targets, features] sized array (features may also be similarities with other targets
    #Y: interaction values, can be real values or binary (+1, -1), insert value float("nan") for unknown entries
    #perfmeasure: function that takes as input a list of correct and predicted outputs, and returns performance
    #higher values should be better, so if using error measures use instead e.g. the inverse -error(Y, P)
    #foldcount: number of cross-validation folds for settings 1-3, setting 4 always runs 3x3 cross-validation


    dataset = DataSet( fpath = FLAGS.dataset_path, ### BUNU ARGS DA GUNCELLE
                      setting_no = FLAGS.problem_type, ##BUNU ARGS A EKLE
                      seqlen = FLAGS.max_seq_len,
                      smilen = FLAGS.max_smi_len,
                      need_shuffle = False )
    # set character set size
    FLAGS.charseqset_size = dataset.charseqset_size
    FLAGS.charsmiset_size = dataset.charsmiset_size

    XD, XT, Y = dataset.parse_data(FLAGS)

    XD = np.asarray(XD)
    XT = np.asarray(XT)
    Y = np.asarray(Y)

    drugcount = XD.shape[0]
    print(drugcount)
    targetcount = XT.shape[0]
    print(targetcount)

    FLAGS.drug_count = drugcount
    FLAGS.target_count = targetcount

    label_row_inds, label_col_inds = np.where(np.isnan(Y)==False)  #basically finds the point address of affinity [x,y]

    if not os.path.exists(figdir):
        os.makedirs(figdir)

    print(FLAGS.log_dir)
    S1_avgperf, S1_avgloss, S1_teststd, trained_model = nfold_1_2_3_setting_sample(XD, XT, Y, label_row_inds, label_col_inds,
                                                                     perfmeasure, deepmethod, FLAGS, dataset)

    logging("Setting " + str(FLAGS.problem_type), FLAGS)
    logging("avg_perf = %.5f,  avg_mse = %.5f, std = %.5f" %
            (S1_avgperf, S1_avgloss, S1_teststd), FLAGS)

    return trained_model

In [17]:
def run_regression( FLAGS ):

    perfmeasure = get_cindex
    deepmethod = build_combined_categorical

    trained_model = experiment(FLAGS, perfmeasure, deepmethod)

    return trained_model

/content/drive/MyDrive/PhazeRo/DeepDTA_Implementation/source

In [18]:
# if __name__=="__main__":

# hyper_params_dict = {'kernel_size': [4, 8, 12],
#                      'num_filters': [32],
#                      'learning_rate': [0.001, 0.0001]}

# paramset1 = FLAGS.num_windows
# paramset2 = FLAGS.smi_window_lengths
# paramset3 = FLAGS.seq_window_lengths

FLAGS = argparser()

FLAGS.num_windows = [32]  # num_filters
FLAGS.smi_window_lengths = [4, 8, 12] #[4,8]  # kernel_size
FLAGS.seq_window_lengths = [0.001, 0.0001] #[8,12]  # will make this the learning_rate just for comparison with the new arch.

FLAGS.batch_size = 256
FLAGS.num_epoch = 1
FLAGS.max_seq_len = 1000
FLAGS.max_smi_len = 100
FLAGS.dataset_path = 'data/davis/'
FLAGS.problem_type = 1
FLAGS.log_dir = 'logs/' + str(time.time()) + "/"
# check for the is_log FLAG
dataset_name = FLAGS.dataset_path.split('/')[1]
if dataset_name == 'davis':
  FLAGS.is_log = 1
else:
  FLAGS.is_log = 0


if not os.path.exists(FLAGS.log_dir):
  os.makedirs(FLAGS.log_dir)

logging(str(FLAGS), FLAGS)
trained_model = run_regression( FLAGS )
trained_model.save('model_deepDTA.keras')


Read data/davis/ start
68
442
logs/1695823136.1194086/
Reading data/davis/ start
val set 5010
train set 20036
val set 5009
train set 20037
val set 5009
train set 20037
val set 5009
train set 20037
val set 5009
train set 20037


Tuning
Number of runs: 30
Number of folds: 5
Number of hyperparameters combinations for each fold: 6

runs_counter: 1/30 --> Tuning.
fold #: 1
NUM_FILTERS: 32
FILTER_LENGTH: 4
learning_rate: 0.001
157/157 [==============================] - 1s 8ms/step

runs_counter: 2/30 --> Tuning.
fold #: 1
NUM_FILTERS: 32
FILTER_LENGTH: 4
learning_rate: 0.0001
157/157 [==============================] - 1s 8ms/step

runs_counter: 3/30 --> Tuning.
fold #: 1
NUM_FILTERS: 32
FILTER_LENGTH: 8
learning_rate: 0.001
157/157 [==============================] - 2s 12ms/step


KeyboardInterrupt: ignored

In [ ]:
trained_model.save('model_deepDTA.keras')

In [ ]:
reconstructed_model = keras.models.load_model("model_deepDTA.keras", custom_objects={"cindex_score": cindex_score})


In [ ]:
reconstructed_model.summary()